In [1]:
import ast
import copy
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sym
from aocd import get_data, submit

DAY = 21
YEAR = 2022

In [2]:
# use test data
raw_test = """root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32"""

# use real data
raw = get_data(day=DAY, year=YEAR)

print(raw_test)

root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32


In [3]:
def parse_data(data):
    data = data.split("\n")
    out = {}
    for d in data:
        objects = re.findall("\w+", d)
        if len(objects) == 2:
            out[objects[0]] = int(objects[1])
        else:
            out[objects[0]] = {"monkeys": [*objects[1:]], "op": re.search("([\+\-\/\*]{1})", d).group(1)}
    return out


dummy = parse_data(raw_test)
real = parse_data(raw)

dummy

{'root': {'monkeys': ['pppw', 'sjmn'], 'op': '+'},
 'dbpl': 5,
 'cczh': {'monkeys': ['sllz', 'lgvd'], 'op': '+'},
 'zczc': 2,
 'ptdq': {'monkeys': ['humn', 'dvpt'], 'op': '-'},
 'dvpt': 3,
 'lfqf': 4,
 'humn': 5,
 'ljgn': 2,
 'sjmn': {'monkeys': ['drzm', 'dbpl'], 'op': '*'},
 'sllz': 4,
 'pppw': {'monkeys': ['cczh', 'lfqf'], 'op': '/'},
 'lgvd': {'monkeys': ['ljgn', 'ptdq'], 'op': '*'},
 'drzm': {'monkeys': ['hmdt', 'zczc'], 'op': '-'},
 'hmdt': 32}

# Part 1

In [4]:
monkeys = copy.deepcopy(real)
numbers = {m: v for m, v in monkeys.items() if isinstance(v, int)}
operations = {m: v for m, v in monkeys.items() if not isinstance(v, int)}

while "root" not in numbers:
    to_del = []
    for m, v in operations.items():
        new_names = [name if name not in numbers else numbers[name] for name in v["monkeys"]]
        if all([isinstance(n, int) for n in new_names]):
            numbers[m] = int(eval(f'{new_names[0]}{v["op"]}{new_names[1]}'))
            to_del.append(m)

    for m in to_del:
        del operations[m]

result = numbers["root"]
result

63119856257960

In [5]:
# submit(result, part="a", day=DAY, year=YEAR)

# Part 2

In [40]:
monkeys = copy.deepcopy(real)

numbers = copy.deepcopy({m: v for m, v in monkeys.items() if isinstance(v, int)})
operations = copy.deepcopy({m: v for m, v in monkeys.items() if not isinstance(v, int)})
x = sym.Symbol("x", real=True)

numbers["humn"] = x

stop = False
while "root" not in numbers:
    to_del = []
    for m, v in operations.items():
        new_names = [name if name not in numbers else numbers[name] for name in v["monkeys"]]
        if all(
            [isinstance(n, float) or isinstance(n, int) or (not isinstance(n, str) and n.is_real) for n in new_names]
        ):
            if m == "root":
                operations[m]["monkeys"] = new_names
                stop = True
                break
            a, b = new_names

            numbers[m] = (
                a + b if v["op"] == "+" else (a - b if v["op"] == "-" else (a * b if v["op"] == "*" else a / b))
            )
            to_del.append(m)

    if stop:
        break

    for m in to_del:
        del operations[m]

a, b = operations["root"]["monkeys"]
result = int(sym.solve(a - b, x)[0])
result

3006709232464

In [ ]:
# submit(result, part="b", day=DAY, year=YEAR)

### Try with bisection

...